In [1]:
!pip install agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.7/976.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.3 MB/s eta 0:00:00


In [2]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=5288b251f21f8ce98e35ae98654ff54a5b3983f71d6d12c84a1d0b26a433939c
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [3]:
import serpapi
print(serpapi.__file__)


/usr/local/lib/python3.11/dist-packages/serpapi/__init__.py


In [4]:
import json
import os
from agno.agent import Agent
from serpapi import GoogleSearch
from agno.tools.serpapi import SerpApiTools
from agno.models.google import Gemini
from datetime import datetime
from agno.models.openrouter import OpenRouter

# --- User Inputs (replace with your desired values for Colab) ---
# You'll need to manually set these variables as there's no UI in Colab
source = "BOM"  # Example: BOM for Mumbai
destination = "DEL"  # Example: DEL for Delhi
num_days = 5
travel_theme = "Couple Getaway"
activity_preferences = "Relaxing on the beach, exploring historical sites"
departure_date = datetime(2025, 10, 26).date()  # Year, Month, Day
return_date = datetime(2025, 10, 31).date()   # Year, Month, Day

# Travel Preferences
budget = "Standard"
flight_class = "Economy"
hotel_rating = "4⭐"

# Packing Checklist (for display/information, not directly used in planning logic here)
packing_list = {
    "👕 Clothes": True,
    "🩴 Comfortable Footwear": True,
    "🕶️ Sunglasses & Sunscreen": False,
    "📖 Travel Guidebook": False,
    "💊 Medications & First-Aid": True
}

# Travel Essentials (for display/information, not directly used in planning logic here)
visa_required = False
travel_insurance = True
currency_converter = False

In [5]:
# --- API Keys ---
# IMPORTANT: Replace with your actual API keys
SERPAPI_KEY = "db2270af85b9be502ee9357b9c6fcf639c8dbb50eba8da99e6cff24724d67e00"
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"


In [6]:
# --- Helper Function ---
def format_datetime(iso_string):
    try:
        dt = datetime.strptime(iso_string, "%Y-%m-%d %H:%M")
        return dt.strftime("%b-%d, %Y | %I:%M %p")  # Example: Mar-06, 2025 | 6:20 PM
    except:
        return "N/A"

# Function to fetch flight data
def fetch_flights(source, destination, departure_date, return_date):
    params = {
        "engine": "Google Flights",
        "departure_id": source,
        "arrival_id": destination,
        "outbound_date": str(departure_date),
        "return_date": str(return_date),
        "currency": "INR",
        "hl": "en",
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

# Function to extract top 3 cheapest flights
def extract_cheapest_flights(flight_data):
    best_flights = flight_data.get("best_flights", [])
    sorted_flights = sorted(best_flights, key=lambda x: x.get("price", float("inf")))[:3]  # Get top 3 cheapest
    return sorted_flights


In [9]:
# --- AI Agents ---
researcher = Agent(
    name="Researcher",
    instructions=[
        "Identify the travel destination specified by the user.",
        "Gather detailed information on the destination, including climate, culture, and safety tips.",
        "Find popular attractions, landmarks, and must-visit places.",
        "Search for activities that match the user’s interests and travel style.",
        "Prioritize information from reliable sources and official travel guides.",
        "Provide well-structured summaries with key insights and recommendations."
    ],
    model=OpenRouter(id="google/gemini-2.5-flash-lite"),
    tools=[SerpApiTools(api_key=SERPAPI_KEY)],
    add_datetime_to_instructions=True,
)

planner = Agent(
    name="Planner",
    instructions=[
        "Gather details about the user's travel preferences and budget.",
        "Create a detailed itinerary with scheduled activities and estimated costs.",
        "Ensure the itinerary includes transportation options and travel time estimates.",
        "Optimize the schedule for convenience and enjoyment.",
        "Present the itinerary in a structured format."
    ],
    model=OpenRouter(id="google/gemini-2.5-flash-lite"),
    add_datetime_to_instructions=True,
)

hotel_restaurant_finder = Agent(
    name="Hotel & Restaurant Finder",
    instructions=[
        "Identify key locations in the user's travel itinerary.",
        "Search for highly rated hotels near those locations.",
        "Search for top-rated restaurants based on cuisine preferences and proximity.",
        "Prioritize results based on user preferences, ratings, and availability.",
        "Provide direct booking links or reservation options where possible."
    ],
    model=OpenRouter(id="google/gemini-2.5-flash-lite"),
    tools=[SerpApiTools(api_key=SERPAPI_KEY)],
    add_datetime_to_instructions=True,
)

In [10]:
# --- Generate Travel Plan ---
print(f"--- Generating Travel Plan for {destination} ({num_days} days) ---")

print("\n✈️ Fetching best flight options...")
flight_data = fetch_flights(source, destination, departure_date, return_date)
cheapest_flights = extract_cheapest_flights(flight_data)

print("\n🔍 Researching best attractions & activities...")
research_prompt = (
    f"Research the best attractions and activities in {destination} for a {num_days}-day {travel_theme.lower()} trip. "
    f"The traveler enjoys: {activity_preferences}. Budget: {budget}. Flight Class: {flight_class}. "
    f"Hotel Rating: {hotel_rating}. Visa Requirement: {visa_required}. Travel Insurance: {travel_insurance}."
)
research_results = researcher.run(research_prompt, stream=False)

print("\n🏨 Searching for hotels & restaurants...")
hotel_restaurant_prompt = (
    f"Find the best hotels and restaurants near popular attractions in {destination} for a {travel_theme.lower()} trip. "
    f"Budget: {budget}. Hotel Rating: {hotel_rating}. Preferred activities: {activity_preferences}."
)
hotel_restaurant_results = hotel_restaurant_finder.run(hotel_restaurant_prompt, stream=False)

print("\n🗺️ Creating your personalized itinerary...")
planning_prompt = (
    f"Based on the following data, create a {num_days}-day itinerary for a {travel_theme.lower()} trip to {destination}. "
    f"The traveler enjoys: {activity_preferences}. Budget: {budget}. Flight Class: {flight_class}. Hotel Rating: {hotel_rating}. "
    f"Visa Requirement: {visa_required}. Travel Insurance: {travel_insurance}. Research: {research_results.content}. "
    f"Flights: {json.dumps(cheapest_flights)}. Hotels & Restaurants: {hotel_restaurant_results.content}."
)
itinerary = planner.run(planning_prompt, stream=False)


--- Generating Travel Plan for DEL (5 days) ---

✈️ Fetching best flight options...

🔍 Researching best attractions & activities...

🏨 Searching for hotels & restaurants...

🗺️ Creating your personalized itinerary...


In [11]:
# --- Display Results ---
print("\n" + "="*50)
print("✈️ Cheapest Flight Options")
if cheapest_flights:
    for idx, flight in enumerate(cheapest_flights):
        airline_name = flight.get("airline", "Unknown Airline")
        price = flight.get("price", "Not Available")
        total_duration = flight.get("total_duration", "N/A")

        flights_info = flight.get("flights", [{}])
        departure = flights_info[0].get("departure_airport", {})
        arrival = flights_info[-1].get("arrival_airport", {})

        departure_time = format_datetime(departure.get("time", "N/A"))
        arrival_time = format_datetime(arrival.get("time", "N/A"))

        # For booking links, you might need to construct them manually or note that they might not be directly available
        # without further API calls or specific search parameters that generate public URLs.
        # The original code's booking link generation might not directly translate to a clickable public URL in all cases.
        booking_link = "Not directly available via this API output"

        print(f"\n--- Flight Option {idx + 1} ---")
        print(f"Airline: {airline_name}")
        print(f"Departure: {departure_time}")
        print(f"Arrival: {arrival_time}")
        print(f"Duration: {total_duration} min")
        print(f"Price: {price}")
        print(f"Booking Link: {booking_link}")
else:
    print("⚠️ No flight data available.")

print("\n" + "="*50)
print("🏨 Hotels & Restaurants")
print(hotel_restaurant_results.content)

print("\n" + "="*50)
print("🗺️ Your Personalized Itinerary")
print(itinerary.content)

print("\n" + "="*50)
print("✅ Travel plan generated successfully!")


✈️ Cheapest Flight Options
⚠️ No flight data available.

🏨 Hotels & Restaurants
Please specify which city you are referring to by 'DEL'.

🗺️ Your Personalized Itinerary
It appears there's been a slight misunderstanding. The IATA airport code "DEL" refers to Indira Gandhi International Airport, which serves **Delhi, India**. Delhi is a vibrant city with a rich history, but it is **not a beach destination**.

Given your interest in "relaxing on the beach," Delhi might not be the ideal location for your getaway.

To help me create the perfect 5-day itinerary for your couple's getaway, please confirm:

1.  **Is Delhi your intended destination?** If so, we can certainly create an amazing historical and cultural itinerary, but we'll need to adjust expectations regarding beach relaxation.
2.  **If not Delhi, what is your desired beach destination?** If you have another city or region in mind that offers both beaches and historical sites, please let me know, and I can research that for you.

